In [1]:
# Duplicate this template, write your scryfall query here and then run the notebook
QUERY = "t:creature is:commander sort:edhrec"
LIMIT = 100

In [2]:
import mtgstats.scryfall.api as api
import mtgstats.cards as c
from mtgstats.analysis import cmc_analysis, color_analysis, subtype_analysis
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [3]:
query = api.search(QUERY, limit=LIMIT)
cards = c.parse_cards(query)

In [4]:
# Go for the Throat - Non-artifact
# Doomblade - non-black
# Malicious Afflitcion - non-black
# Victim of Night - non-Vampire, non-Werewolf, non-Zombie
# Vendetta - non-black, can't regenerate, loses life equal to toughness
# Terror - Non-black
# Ultimate Price - Monocolored
# Devour in shadow - Can't regenerate, loses life equal to toughness
# Power Word Kill - non-Angel, non-Demon, non-Devil, non-Dragon

In [5]:
destructible = list(filter(lambda x: "Indestructible" not in x.get('keywords', []), cards))
c = color_analysis(destructible)
cmc, cum_cmc = cmc_analysis(destructible)
types = subtype_analysis(destructible)
data = pd.DataFrame(destructible)

Nissa, Vastwood Seer


In [6]:
# Número de criaturas não pretas: Doomblade, Malicious Affliction, Vendetta
non_black_creatures = len(data) - c['B']
print(non_black_creatures)

62


In [7]:
artifact_creatures =  len(data[data['type_line'].str.contains("Artifact")])
# Número de criaturas artefatos
print(len(data) - artifact_creatures)
# Terror
print(non_black_creatures - artifact_creatures)

84
61


In [8]:
# Numero de criaturas monocoloridas: Ultimate price
len(data[data.colors.str.len() == 1])

58

In [9]:
# Victim of Night
t = ['Zombie', 'Werewolf', 'Vampire']
len(data) - sum([types.get(x, 0) for x in t])

75

In [10]:
# Power Word Kill
t = ['Angel', 'Demon', 'Devil', 'Dragon']
len(data) - sum([types.get(x, 0) for x in t])

74

In [11]:
def tough_helper(x):
    t = x.get('toughness', 0)
    # Cartas com * na resistência vão ficar marcadas com -1
    return int(t) if t != '*' else 0

In [12]:
# Distribuição das resistências para Vendetta (criaturas não pretas)
t = list(
    map(tough_helper, 
        filter(lambda x: 'B' not in x.get('colors', []),
               destructible)))
from collections import Counter
counter = Counter(t)
s = sorted(counter.items(), key=lambda x:x[0])

_, t = zip(*s)
# Soma cumulativa <resitencia>: <soma>
dict(zip(_,np.cumsum(t)))

{0: 2, 1: 5, 2: 16, 3: 28, 4: 43, 5: 49, 6: 54, 7: 58, 12: 61, 13: 62}

In [13]:
# Distribuição das resistências para Malicious Afliction
t = list(
    map(tough_helper,destructible))
from collections import Counter
counter = Counter(t)
s = sorted(counter.items(), key=lambda x:x[0])

_, t = zip(*s)
# Soma cumulativa <resitencia>: <soma>
dict(zip(_,np.cumsum(t)))

{0: 2, 1: 5, 2: 20, 3: 36, 4: 56, 5: 65, 6: 75, 7: 79, 8: 81, 12: 84, 13: 85}